<a href="https://colab.research.google.com/github/magemongo/ContentLovers/blob/master/Scrapping_Publisher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loop 1 - lista de links do portal


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from urllib.request import Request, urlopen

req = Request('https://www.guiademidia.com.br/jornais.htm', headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, 'lxml')
type(soup)

In [ ]:
As = soup.find_all('a')

In [ ]:
As

[<a href="https://www.guiademidia.com.br"> <img alt="Logo Site Guia de Mídia" src="imagens/logo/v7/logo-guia-de-midia-120x72.png"/></a>,
 <a class="letra-cor3" href="https://www.guiademidia.com.br">home</a>,
 <a class="vermelhao" href="https://www.guiademidia.com.br/jornaisdesaopaulo.htm">Jornais de SP</a>,
 <a class="letra-cor6" href="https://www.guiademidia.com.br/jornaisdoriodejaneiro.htm">Jornais do Rio</a>,
 <a href="https://www.guiademidia.com.br/jornaisdoacre.htm" title="Jornais Acreanos">Jornais do Acre</a>,
 <a href="https://www.guiademidia.com.br/acre/jornais-de-rio-branco.htm" title="Jornais de Rio Branco AC">Rio Branco</a>,
 <a href="https://www.guiademidia.com.br/jornaisdealagoas.htm" title="Jornais Alagoanos">Jornais de Alagoas</a>,
 <a href="https://www.guiademidia.com.br/alagoas/jornais-de-maceio.htm" title="Jornais de Maceió AL">Maceió</a>,
 <a href="https://www.guiademidia.com.br/jornaisdoamapa.htm" title="Jornais Amapaenses">Jornais do Amapá</a>,
 <a href="https://ww

In [ ]:
first = soup.find(text="Jornais do Rio")

In [ ]:
jb_links = first.find_all_next('a')

In [ ]:
links_jb = []
for a in jb_links:
  if a.get('href')!= 'agenciasdenoticias.htm':
    links_jb.append(a.get('href'))
  else:
    break

In [ ]:
links_jb_estado = []
links_jb_capital = []
links_sp_capital = []

for link in links_jb:
  if link.count('/') == 3:
    links_jb_estado.append(link)
  elif link.count('/') == 4:
    links_jb_capital.append(link)
  else:
    links_sp_capital.append(link)

# Scrapping Jornais Brasileiros por Estado
lista = links_jb_estado

In [ ]:
import re

c = 0
dados = {}

dados['nome'] = []
dados['link'] = []

for link in links_jb_estado:
  req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
  c += 1
  print(req, c)

  webpage = urlopen(req).read()
  soup = BeautifulSoup(webpage, 'lxml')
  rows = soup.find_all('tr')

  for row in rows:
      cells = row.find_all('td')
      links = row.find_all('a')
      str_links = str(links)
      str_cells = str(cells)
      clean = re.compile('<.*?>')
      clean_l = re.compile("((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*")
      clean3 = (re.search(clean_l, str_links))
      clean2 = (re.sub(clean, '',str_cells))
      dados['nome'].append(clean2.rstrip('[]'))
      pre_link = str(clean3).split('match=')
      if len(pre_link) > 1:
        dados['link'].append(pre_link[1].rstrip('>'))
      else:
        dados['link'].append(pre_link[0].rstrip('None'))
  type(clean2)


<urllib.request.Request object at 0x7fe29ba57048> 1
<urllib.request.Request object at 0x7fe29a4b4be0> 2
<urllib.request.Request object at 0x7fe29ba57128> 3
<urllib.request.Request object at 0x7fe29a7b4c50> 4
<urllib.request.Request object at 0x7fe29b767d30> 5
<urllib.request.Request object at 0x7fe29b021e80> 6
<urllib.request.Request object at 0x7fe29aca2160> 7
<urllib.request.Request object at 0x7fe29aabbf98> 8
<urllib.request.Request object at 0x7fe29a932588> 9
<urllib.request.Request object at 0x7fe29b9a6f28> 10
<urllib.request.Request object at 0x7fe29aabbeb8> 11
<urllib.request.Request object at 0x7fe29aca2240> 12
<urllib.request.Request object at 0x7fe29a345080> 13
<urllib.request.Request object at 0x7fe29a33c668> 14
<urllib.request.Request object at 0x7fe29a932e10> 15
<urllib.request.Request object at 0x7fe29b302b00> 16
<urllib.request.Request object at 0x7fe29b302ef0> 17
<urllib.request.Request object at 0x7fe29b08e940> 18
<urllib.request.Request object at 0x7fe29b08ea20> 19
<u

In [ ]:
df = pd.DataFrame(dados)
df.tail()

,nome,link
3758,"[Blog Jaciara Barros | Coluna Social, Palmas",'http://www.jaciarabarros.com.br/'
3759,"[Blog Palmas Aqui | Antônio Guimarães, Palmas",'https://palmasaqui.wordpress.com/'
3760,[Blog Cidade Interativa Almas TO | Rinaldo M. ...,'http://cialmas.blogspot.com.br/'
3761,"[Blog do Luiz Martins, Guaraí",'http://www.luizmartins.net/'
3762,"[ , + Blogs",'https://www.guiademidia.com.br/blogs/jornalis...


In [ ]:
import numpy as np

df.replace('', np.nan, inplace=True)
df.dropna(axis=0, inplace=True)

In [ ]:
df

,nome,link
2,"[A Gazeta do Acre, Rio Branco",'http://www.agazetadoacre.com'
3,"[Jornal Página 20, Rio Branco",'http://www.pagina20.net'
4,"[O Rio Branco, Rio Branco",'http://www.oriobranco.net/'
5,"[A Tribuna do Acre, Rio Branco",'http://www.jornalatribuna.com.br/'
6,"[Jornal Opinião, Rio Branco",'http://www.jornalopiniao.net/'
...,...,...
3758,"[Blog Jaciara Barros | Coluna Social, Palmas",'http://www.jaciarabarros.com.br/'
3759,"[Blog Palmas Aqui | Antônio Guimarães, Palmas",'https://palmasaqui.wordpress.com/'
3760,[Blog Cidade Interativa Almas TO | Rinaldo M. ...,'http://cialmas.blogspot.com.br/'
3761,"[Blog do Luiz Martins, Guaraí",'http://www.luizmartins.net/'


In [ ]:
df.to_excel('/content/publishers_estados.xls')

# Scrapping Jornais Brasileiros por Capital

lista = links_jb_capital

In [ ]:
for link in links_jb_capital:
  req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
  c += 1
  print(req, c)

  webpage = urlopen(req).read()
  soup = BeautifulSoup(webpage, 'lxml')
  rows = soup.find_all('tr')

In [ ]:
req = Request('https://www.guiademidia.com.br/acre/jornais-de-rio-branco.htm', headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, 'lxml')
type(soup)

bs4.BeautifulSoup

In [ ]:
listas = soup.find_all("div", class_="lista")


In [ ]:
dados2 = {}
dados2['nomes'] = []
pre_nomes = []

for lista in listas:
  cells = lista.find_all('a')
  str_cells = str(cells)
  clean = re.compile('<.*?>')
  clean2 = (re.sub(clean, '',str_cells))
  pre_nomes.append(clean2)

dados2['nomes'] = pre_nomes[0] + pre_nomes[1]

<_sre.SRE_Match object; span=(24, 75), match='https://www.guiademidia.com.br/blogs/jornalismo.h>
<_sre.SRE_Match object; span=(24, 75), match='https://www.guiademidia.com.br/blogs/jornalismo.h>


In [ ]:
dados2 = {}
dados2['nomes'] = []
pre_nomes = []

for lista in listas:
  cells = lista.find_all('a')
  str_cells = str(cells)
  str_links = str(cells)
  clean_l = re.compile("((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*")
  clean3 = (re.search(clean_l, str_links))
  clean = re.compile('<.*?>')
  clean2 = (re.sub(clean, '',str_cells))
  pre_nomes.append(clean2)
  print(clean3)

dados2['nomes'] = pre_nomes[0] + pre_nomes[1]

<_sre.SRE_Match object; span=(10, 36), match='https://jornalopiniao.net/'>
<_sre.SRE_Match object; span=(10, 47), match='https://ambienteacreano.blogspot.com/'>


In [ ]:
listas[0].find_all('a', href=True)


[<a href="https://jornalopiniao.net/" target="_blank">Jornal Opinião</a>,
 <a href="http://www.oriobranco.net/" target="_blank">Jornal O Rio Branco</a>,
 <a href="https://agazetadoacre.com/" target="_blank">A Gazeta do Acre</a>,
 <a href="https://pagina20.net/" target="_blank">Jornal Página 20</a>,
 <a href="http://www.jornalatribuna.com.br/" target="_blank">Jornal A tribuna do Acre</a>,
 <a href="https://acmanchete.com/" target="_blank">AC Manchete</a>,
 <a href="https://acreaovivo.com.br/" target="_blank">Acre Ao Vivo</a>,
 <a href="https://contilnetnoticias.com.br/" target="_blank">Contilnet Notícias</a>,
 <a href="https://ecosdanoticia.net.br/" target="_blank">Ecos da Notícia</a>,
 <a href="https://www.ac24horas.com/" target="_blank">AC 24 Horas</a>,
 <a href="https://www.acjornal.com/" target="_blank">AC Jornal</a>,
 <a href="http://acrealerta.com" target="_blank">Acre Alerta</a>,
 <a href="https://acreemrevista.com.br/" target="_blank">Acre em Revista</a>,
 <a href="https://www.a